**Question 1:** What is TensorFlow Object Detection API (TFOD2) and what are its
primary components?

  - TensorFlow Object Detection API (TFOD2) is an open-source framework by Google used to train, evaluate, and deploy object detection models easily. It is widely used in Google Colab for quick setup and GPU-based training.

**Primary Components:**

**1. Model Zoo**

Pre-trained models (SSD, Faster R-CNN, EfficientDet) to start quickly.

**2. Pipeline Config File**

.config file to set model, dataset paths, batch size, learning rate, etc.

**3. TFRecord Dataset**

Optimized data format used for training and evaluation.

**4. Training Script**

model_main_tf2.py used to train the model in Colab.

**5. Evaluation Script**

To check model accuracy (mAP) during/after training.

**6. Export / Inference Tools**

Export trained model for prediction on new images/videos.

---

**Question 2:** Differentiate between semantic segmentation and instance segmentation. Provide examples of where each might be used.

-  **Semantic Segmentation vs Instance Segmentation:**

| Feature         | Semantic Segmentation                  | Instance Segmentation              |
| --------------- | -------------------------------------- | ---------------------------------- |
| What it does    | Labels **each pixel by class**         | Labels **each object separately**  |
| Object identity | ❌ Does NOT separate same-class objects | ✅ Separates each individual object |
| Output          | Class mask                             | Mask + object ID                   |
| Example         | All cars = same label                  | Each car = different mask          |
| Common Models   | U-Net, DeepLab                         | Mask R-CNN                         |


**Examples:**

**Semantic Segmentation:**

 - Road vs sky vs building in self-driving

 - Medical image (tumor vs normal tissue)

 - Land-use classification in satellite images

**Instance Segmentation:**

 - Counting people in a crowd

 - Separating multiple cells in microscopy

 - Detecting each car/person in surveillance

---

**Question 3:** Explain the Mask R-CNN architecture. How does it extend Faster R-CNN?

 - **Mask R-CNN Architecture:**

Mask R-CNN is an extension of Faster R-CNN for instance segmentation.

- **How it extends Faster R-CNN:**

1. Uses same Backbone + RPN for region proposals

2. Replaces RoIPool with RoIAlign for better pixel accuracy

3. Adds a new Mask Head (parallel branch)

4. Predicts 3 outputs per object:

- Class label

- Bounding box

- Pixel-level mask

---

**Question 4:** Describe the purpose of masks in image segmentation. How are they used during training and inference?

 - Mask is a pixel-level map that shows which pixels belong to which object or class.

**Purpose of Masks:**

 - Separate object from background

 - Show exact shape and area of objects

 - Enable pixel-level understanding

 - Used in medical, autonomous driving, satellite images, etc.

**During Training:**

 - Masks are used as ground truth labels

 - Model learns to match predicted masks with true masks

 - Mask loss (e.g., binary cross-entropy) is calculated

**During Inference:**

 - Model predicts masks for new images

 - Masks are overlaid on image

 - Used to visualize and measure object area/shape

 ---

**Question 5:** What are the steps involved in training a custom image segmentation model using TFOD2?

 - **Steps to Train a Custom Image Segmentation Model using TFOD2**

**Collect & Label Data**

Annotate images with boxes + masks (using LabelImg, CVAT, LabelMe, etc.)

**Convert to TFRecord**

Convert annotations to TFRecord format for TFOD2.

**Choose Pre-trained Model**

Download a model from TFOD2 Model Zoo (e.g., Mask R-CNN).

**Edit Pipeline Config Set:**

 - Number of classes

 - Dataset paths

 - Batch size, learning rate

**Train the Model**

Run model_main_tf2.py in Google Colab (with GPU).

**Evaluate Model**

Check metrics like mAP and mask accuracy.

**Export Trained Model**

Export for inference.

**Run Inference**

Test on new images/videos

---


**Question 6:** Write a Python script to install TFOD2 and verify its installation by printing the available model configs.

In [ ]:
# Install TFOD2 (One-cell Google Colab Script)

!git clone https://github.com/tensorflow/models.git
!apt-get install -y protobuf-compiler
!pip install -U pip
!pip install tensorflow tf_slim pycocotools

# Compile protos
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.

# Set PYTHONPATH
import os, sys
sys.path.append(os.path.abspath('.'))
sys.path.append(os.path.abspath('./slim'))

# Test installation
!python object_detection/builders/model_builder_tf2_test.py

# Print available TFOD2 model configs
!ls object_detection/configs/tf2/


**Question 7:** Create a Python script to load a labeled dataset (in TFRecord format) and visualize the annotation masks over the images.


In [ ]:
# ===== Visualize Masks from TFRecord (Single Script) =====

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

TFRECORD_PATH = "/content/train.record"   # change path

# Feature description (common TFOD2 format)
feature_description = {
    'image/encoded': tf.io.FixedLenFeature([], tf.string),
    'image/height': tf.io.FixedLenFeature([], tf.int64),
    'image/width': tf.io.FixedLenFeature([], tf.int64),
    'image/object/mask': tf.io.VarLenFeature(tf.string),
}

def _parse_function(example_proto):
    example = tf.io.parse_single_example(example_proto, feature_description)
    image = tf.image.decode_jpeg(example['image/encoded'], channels=3)
    masks = tf.sparse.to_dense(example['image/object/mask'], default_value=b'')
    return image, masks

raw_dataset = tf.data.TFRecordDataset(TFRECORD_PATH)
parsed_dataset = raw_dataset.map(_parse_function)

# Visualize first sample
for image, masks in parsed_dataset.take(1):
    img = image.numpy()
    plt.figure(figsize=(6,6))
    plt.imshow(img)

    # Overlay masks
    for m in masks.numpy():
        if len(m) > 0:
            mask = tf.image.decode_png(m, channels=1).numpy()
            plt.imshow(mask.squeeze(), alpha=0.4, cmap='jet')

    plt.title("Image with Annotation Masks")
    plt.axis('off')
    plt.show()


**Question 8:**  Using a pre-trained Mask R-CNN model, write a code snippet to perform inference on a single image and plot the predicted masks.

In [ ]:
# ===== Mask R-CNN Inference + Plot Masks (Single Script) =====

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

# Paths (change as needed)
PIPELINE_CONFIG = "/content/models/research/object_detection/configs/tf2/mask_rcnn_resnet50_v1_fpn_640x640_coco17_tpu-8.config"
MODEL_DIR = "/content/pretrained_model"
LABEL_MAP = "/content/mscoco_label_map.pbtxt"
IMAGE_PATH = "/content/test.jpg"

# Load saved model
detect_fn = tf.saved_model.load(MODEL_DIR + "/saved_model")

# Load image
img = tf.io.read_file(IMAGE_PATH)
img = tf.image.decode_jpeg(img, channels=3)
input_tensor = tf.expand_dims(img, 0)

# Run inference
detections = detect_fn(input_tensor)

# Convert to numpy
num = int(detections.pop('num_detections'))
detections = {k: v[0, :num].numpy() for k, v in detections.items()}
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

# Load label map
category_index = label_map_util.create_category_index_from_labelmap(
    LABEL_MAP, use_display_name=True)

# Visualize results
image_np = img.numpy()
viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np,
    detections['detection_boxes'],
    detections['detection_classes'],
    detections['detection_scores'],
    category_index,
    instance_masks=detections.get('detection_masks_reframed', None),
    use_normalized_coordinates=True,
    min_score_thresh=0.5
)

plt.figure(figsize=(8,8))
plt.imshow(image_np)
plt.title("Mask R-CNN Predicted Masks")
plt.axis('off')
plt.show()


**Question 9:** Write a Python script to evaluate a trained TFOD2 Mask R-CNN model and plot the Precision-Recall curve.

In [ ]:
# ===== TFOD2 Mask R-CNN Evaluation + Precision-Recall Curve =====

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from object_detection.metrics import coco_evaluation
from object_detection.utils import label_map_util

# Paths (change as needed)
PIPELINE_CONFIG = "/content/pipeline.config"
CHECKPOINT_DIR = "/content/training"
LABEL_MAP = "/content/label_map.pbtxt"
TFRECORD_VAL = "/content/val.record"

# Load label map
category_index = label_map_util.create_category_index_from_labelmap(
    LABEL_MAP, use_display_name=True)

# COCO Evaluator (for boxes + masks)
categories = list(category_index.values())
evaluator = coco_evaluation.CocoMaskEvaluator(categories)

# Dummy example loop (structure for PR calculation)
# NOTE: In real setup, TFOD2 eval script feeds GT + predictions automatically

precisions = []
recalls = []

# Example (mock values for assignment-style plotting)
# Normally, these come from COCO evaluation outputs
precisions = [0.9, 0.85, 0.8, 0.7, 0.6]
recalls    = [0.1, 0.3, 0.5, 0.7, 0.9]

# Plot Precision-Recall Curve
plt.figure()
plt.plot(recalls, precisions, marker='o')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve (Mask R-CNN)")
plt.grid(True)
plt.show()


**Question 10:** You are working with a city surveillance team to identify illegal parking zones from street camera images. The model you built detects cars using bounding boxes, but the team reports inaccurate overlaps with sidewalks and fails in complex street scenes.How would you refine your model to improve accuracy, especially around object boundaries? What segmentation strategy and tools would you use?

In [ ]:
# ===== Boundary-Accurate Illegal Parking Detection (Single Script) =====
# Strategy: Use Mask R-CNN (Instance Segmentation) instead of only boxes

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import label_map_util

# ---------------- PATHS ----------------
PIPELINE_CONFIG = "/content/pipeline.config"
MODEL_DIR = "/content/training"
LABEL_MAP = "/content/label_map.pbtxt"
IMAGE_PATH = "/content/street.jpg"

# ---------------- TRAIN (Better Boundaries) ----------------
# Mask R-CNN with pixel-level masks for car, sidewalk, road
print("Training Mask R-CNN for precise object boundaries...")

!python models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path=/content/pipeline.config \
  --model_dir=/content/training \
  --alsologtostderr

# ---------------- INFERENCE (Check Overlap with Sidewalk) ----------------
print("Running inference with instance masks...")

detect_fn = tf.saved_model.load(MODEL_DIR + "/exported_model/saved_model")

# Load image
img = tf.io.read_file(IMAGE_PATH)
img = tf.image.decode_jpeg(img, channels=3)
input_tensor = tf.expand_dims(img, 0)

# Run model
detections = detect_fn(input_tensor)

num = int(detections.pop('num_detections'))
detections = {k: v[0, :num].numpy() for k, v in detections.items()}
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

# Load labels
category_index = label_map_util.create_category_index_from_labelmap(
    LABEL_MAP, use_display_name=True)

image_np = img.numpy()

# Visualize precise masks (for sidewalk overlap accuracy)
viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np,
    detections['detection_boxes'],
    detections['detection_classes'],
    detections['detection_scores'],
    category_index,
    instance_masks=detections.get('detection_masks_reframed', None),
    min_score_thresh=0.5,
    use_normalized_coordinates=True
)

plt.figure(figsize=(8,8))
plt.imshow(image_np)
plt.title("Mask R-CNN: Precise Car vs Sidewalk Boundaries")
plt.axis('off')
plt.show()

print("Done: Instance segmentation used for accurate object boundaries.")
